In [1]:
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE, KMeansSMOTE, SVMSMOTE
from imblearn.under_sampling import CondensedNearestNeighbour, EditedNearestNeighbours,  \
                                    RepeatedEditedNearestNeighbours, AllKNN, NearMiss, NeighbourhoodCleaningRule, OneSidedSelection, TomekLinks
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score 
from imblearn.combine import SMOTETomek, SMOTEENN

import random
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('../datasets/BankChurners.csv')
list = ['Attrition_Flag', 'Total_Trans_Ct', 'Total_Trans_Amt', 'Total_Revolving_Bal', 'Total_Ct_Chng_Q4_Q1', 'Contacts_Count_12_mon', 'Total_Relationship_Count', 'Months_Inactive_12_mon', 'Months_on_book']
data = data[list]
data

,Attrition_Flag,Total_Trans_Ct,Total_Trans_Amt,Total_Revolving_Bal,Total_Ct_Chng_Q4_Q1,Contacts_Count_12_mon,Total_Relationship_Count,Months_Inactive_12_mon,Months_on_book
0,Existing Customer,42,1144,777,1.625,3,5,1,39
1,Existing Customer,33,1291,864,3.714,2,6,1,44
2,Existing Customer,20,1887,0,2.333,0,4,1,36
3,Existing Customer,20,1171,2517,2.333,1,3,4,34
4,Existing Customer,28,816,0,2.500,0,5,1,21
...,...,...,...,...,...,...,...,...,...
10122,Existing Customer,117,15476,1851,0.857,3,3,2,40
10123,Attrited Customer,69,8764,2186,0.683,3,4,2,25
10124,Attrited Customer,60,10291,0,0.818,4,5,3,36
10125,Attrited Customer,62,8395,0,0.722,3,4,3,36


In [3]:
object_columns = data.select_dtypes('object').columns
for i in object_columns:
    lb = LabelEncoder()
    lb.fit(data[i])
    data[i] = lb.transform(data[i])
    print(f'category : {np.unique(data[i])}\nclasses : {lb.classes_}\n')

input = data.iloc[:,1:]
target = data.iloc[:,0]

category : [0 1]
classes : ['Attrited Customer' 'Existing Customer']



In [4]:
## Sampling 전부

columns = ['f1_score', 'recall', 'precision', 'accuracy']
df = pd.DataFrame(columns=columns)

sampling = [EditedNearestNeighbours,RepeatedEditedNearestNeighbours, AllKNN, NearMiss,NeighbourhoodCleaningRule, OneSidedSelection,TomekLinks, SMOTE, BorderlineSMOTE, ADASYN, SMOTETomek, SMOTEENN]

random.seed(42)

for i in sampling:
    x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

    ss = StandardScaler()
    x_train_ss = ss.fit_transform(x_train)
    x_test_ss = ss.fit_transform(x_test)

    sampling = i()
    x_train_ss, y_train = sampling.fit_resample(x_train_ss, y_train)

    xgb = XGBClassifier(random_state=42)
    xgb.fit(x_train_ss, y_train)
    pred = xgb.predict(x_test_ss)

    acc = accuracy_score(y_test, pred)
    recall = recall_score(y_test, pred)
    precision = precision_score(y_test, pred)
    f1 = f1_score(y_test, pred)

    df.loc[sampling] = [f1, recall, precision, acc]

print("XGB Model's score by sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False, inplace=True)
df.head(10)

XGB Model's score by sampling


,f1_score,recall,precision,accuracy
TomekLinks(),0.976009,0.981754,0.970332,0.959526
OneSidedSelection(),0.975051,0.977634,0.972482,0.958045
NeighbourhoodCleaningRule(),0.974919,0.972337,0.977515,0.958045
EditedNearestNeighbours(),0.971768,0.962331,0.981393,0.953110
AllKNN(),0.971174,0.961742,0.980792,0.952122
RepeatedEditedNearestNeighbours(),0.968647,0.954679,0.983030,0.948174
SMOTEENN(),0.923368,0.861683,0.994565,0.880059
NearMiss(),0.918851,0.866392,0.978073,0.871668
SMOTETomek(),0.840273,0.724544,1.000000,0.769003
BorderlineSMOTE(),0.834705,0.716304,1.000000,0.762093


In [5]:
## Under Sampling 만

columns = ['f1_score', 'recall', 'precision', 'accuracy']
df = pd.DataFrame(columns=columns)

sampling = [EditedNearestNeighbours, RepeatedEditedNearestNeighbours, AllKNN, NearMiss,NeighbourhoodCleaningRule, OneSidedSelection,TomekLinks]

random.seed(42)

for i in sampling:
    
    x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

    ss = StandardScaler()
    x_train_ss = ss.fit_transform(x_train)
    x_test_ss = ss.fit_transform(x_test)

    sampling = i()
    x_train_ss, y_train = sampling.fit_resample(x_train_ss, y_train)

    xgb = XGBClassifier(random_state=42)
    xgb.fit(x_train_ss, y_train)
    pred = xgb.predict(x_test_ss)

    acc = accuracy_score(y_test, pred)
    recall = recall_score(y_test, pred)
    precision = precision_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    
    df.loc[sampling] = [f1, recall, precision, acc]

print("XGB Model's score by Under sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False)

XGB Model's score by Under sampling


,f1_score,recall,precision,accuracy
OneSidedSelection(),0.977452,0.982343,0.972611,0.961994
TomekLinks(),0.976009,0.981754,0.970332,0.959526
NeighbourhoodCleaningRule(),0.974919,0.972337,0.977515,0.958045
EditedNearestNeighbours(),0.971768,0.962331,0.981393,0.953110
AllKNN(),0.971174,0.961742,0.980792,0.952122
RepeatedEditedNearestNeighbours(),0.968647,0.954679,0.983030,0.948174
NearMiss(),0.918851,0.866392,0.978073,0.871668


In [6]:
## Over Sampling 만

columns = ['f1_score', 'recall', 'precision', 'accuracy']
df = pd.DataFrame(columns=columns)

sampling = [SMOTE, BorderlineSMOTE, ADASYN]
strategy_ = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

random.seed(42)

for i in sampling:
    
    for strategy in strategy_:
        x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

        ss = StandardScaler()
        x_train_ss = ss.fit_transform(x_train)
        x_test_ss = ss.fit_transform(x_test)

        sampling = i(sampling_strategy=strategy)
        x_train_ss, y_train = sampling.fit_resample(x_train_ss, y_train)

        xgb = XGBClassifier(random_state=42)
        xgb.fit(x_train_ss, y_train)
        pred = xgb.predict(x_test_ss)

        acc = accuracy_score(y_test, pred)
        recall = recall_score(y_test, pred)
        precision = precision_score(y_test, pred)
        f1 = f1_score(y_test, pred)
        
        df.loc[i.__name__+'('+str(strategy)+')'] = [f1, recall, precision, acc]

print("XGB Model's score by Over sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False, inplace=True)
df.head(10)

XGB Model's score by Over sampling


,f1_score,recall,precision,accuracy
SMOTE(0.3),0.973404,0.969394,0.977448,0.955577
SMOTE(0.4),0.961109,0.938199,0.985167,0.936328
ADASYN(0.3),0.957234,0.928782,0.987484,0.930405
SMOTE(0.5),0.953020,0.919364,0.989234,0.923988
BorderlineSMOTE(0.3),0.952902,0.922896,0.984925,0.923495
BorderlineSMOTE(0.4),0.939864,0.892290,0.992796,0.904245
SMOTE(0.6),0.937811,0.887581,0.994067,0.901283
SMOTE(0.7),0.915049,0.846380,0.995845,0.868213
SMOTE(0.8),0.910948,0.839906,0.995119,0.862290
BorderlineSMOTE(0.6),0.899645,0.820483,0.995714,0.846496


In [7]:
## Combine Sampling 만

columns = ['f1_score', 'recall', 'precision', 'accuracy']
df = pd.DataFrame(columns=columns)

sampling = [SMOTETomek, SMOTEENN]

random.seed(42)

for i in sampling:

    x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

    ss = StandardScaler()
    x_train_ss = ss.fit_transform(x_train)
    x_test_ss = ss.fit_transform(x_test)

    sampling = i()
    x_train_ss, y_train = sampling.fit_resample(x_train_ss, y_train)

    xgb = XGBClassifier(random_state=42)
    xgb.fit(x_train_ss, y_train)
    pred = xgb.predict(x_test_ss)

    acc = accuracy_score(y_test, pred)
    recall = recall_score(y_test, pred)
    precision = precision_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    
    df.loc[sampling] = [f1, recall, precision, acc]

print("XGB Model's score by Combine sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False)

XGB Model's score by Combine sampling


,f1_score,recall,precision,accuracy
SMOTEENN(),0.931250,0.876986,0.992672,0.891412
SMOTETomek(),0.884956,0.794585,0.998521,0.826752
